In [1]:
from IPython.display import display
from ipywidgets import Image, HBox, Dropdown, VBox, SelectionSlider, Layout
import cv2


In [2]:
from pathlib import Path

In [3]:
from timelapse import Timeline

In [4]:
AQARAPATH = "/mnt/cameras/aqara_video"

In [5]:
mapping_cameras = {
    "lumi1.54ef44457bc9": "Pasto",
    "lumi1.54ef44603857": "Living",
}

In [6]:
img_widget = Image()

In [13]:
cameras = [(mapping_cameras.get(c.name, c.name), c) for c in Path(AQARAPATH).glob("lumi1.*")]
dropdown_cameras = Dropdown(
    options=cameras,
    description='Camera:',
)
dropdown_days = Dropdown(description='Day:')
selector_minutes = SelectionSlider(options=[('No videos available', "")], description='Minute:', disabled=True, layout=Layout(width='1000px'))
clips = []

def update_days(*args):
    global clips
    clips_path = dropdown_cameras.value
    clips = Timeline(clips_path).clips

    days = sorted(list(set(c.timestamp.date() for c in clips)))
    dropdown_days.options = days
    # Also update minutes when days change
    update_minutes()

def update_minutes(*args):
    if dropdown_days.value is not None:
        selected_day = dropdown_days.value
        minutes = [(c.timestamp.strftime("%H:%M:%S"), c.path) for c in clips if c.timestamp.date() == selected_day]
        minutes = sorted(minutes)

        if minutes:  # Only update if we have options
            selector_minutes.options = minutes
            selector_minutes.disabled = False
        else:
            selector_minutes.options = []
            selector_minutes.disabled = True

def update_video(*args):
    if selector_minutes.value is not None:
        cap = cv2.VideoCapture(str(selector_minutes.value))
        try:
            ret, frame = cap.read()
            if ret:
                _, jpeg = cv2.imencode('.jpg', frame)
                img_widget.value = jpeg.tobytes()
        finally:
            cap.release()
            
dropdown_cameras.observe(update_days, names='value')
dropdown_days.observe(update_minutes, names='value')
selector_minutes.observe(update_video, names='value')

# Initial update to show something
update_days()


In [14]:
display(
    VBox([dropdown_cameras, dropdown_days, selector_minutes, 
          HBox([img_widget])]
        )

)

In [ ]:

cap = cv2.VideoCapture(str(selector_minutes.value))

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        _, jpeg = cv2.imencode('.jpg', frame)

        img_widget.value = jpeg.tobytes()

except KeyboardInterrupt:
    print("Interrupted")

cap.release()
    
